In [1]:
import pandas as pd
import numpy as np
articles = pd.read_csv('articles.csv')
articles.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [2]:
articles.shape

(167053, 6)

In [3]:
articles.title[0]

"Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral"

# Sentencepiece (Words vocab)

In [3]:
!pip3 install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.0 MB 494 kB/s eta 0:00:01


In [4]:
import sentencepiece as spm
import re

In [5]:
#titles = articles.title.apply(lambda x: re.sub('[\t]', '', x)).str.cat(sep=' ')
titles = articles.title.apply(lambda x: re.sub('[\t]', '', '<s> '+ x + '</s>')).str.cat(sep=' ')

In [6]:
len(titles)

11725850

In [6]:
titles_file = open('titles.txt', 'w+')
titles_file.write(titles)
titles_file.close()

In [7]:
spm.SentencePieceTrainer.train('--input=titles.txt --model_prefix=m --user_defined_symbols=<s>,</s> --vocab_size=8000')

True

In [8]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [9]:
# text => id
print(sp.encode_as_pieces(' <s> Brasil econômia</s>'))
print(sp.encode_as_ids('<s> Brasil econômia</s>'))

# id => text
print(sp.decode_pieces(['▁Brasil', '▁e', 'con', 'ômi', 'a']))
print(sp.decode_ids([34, 6, 714, 5195, 14]))

['▁', '<s>', '▁Brasil', '▁e', 'con', 'ômi', 'a', '</s>']
[3, 1, 37, 7, 685, 4031, 14, 2]
Brasil econômia
se, Russomanno confirmadoa


In [10]:
sp.id_to_piece(1) == '<s>'

True

In [11]:
print(sp.get_piece_size())

# id <=> piece conversion
print(sp.id_to_piece(209))
print(sp.piece_to_id('í'))

print(sp.piece_to_id('__MUST_BE_UNKNOWN__'))

# <unk>, <s>, </s> are defined by default. Their ids are (0, 1, 2)
# <s> and </s> are defined as 'control' symbol.
for id in range(3):
  print(sp.id_to_piece(id), sp.is_control(id))

8000
▁TV
161
0
<unk> False
<s> False
</s> False


# Input Transform

In [12]:
import tensorflow as tf

In [13]:
titles_as_id = []
for title in titles.split(sep='</s>')[:-1]:
    titles_as_id+=sp.encode_as_ids(title)
    titles_as_id.append(sp.piece_to_id('</s>'))
len(titles_as_id)

3074017

In [14]:
sp.decode_ids([8, 1, 132, 23, 20, 146, 18, 1569, 120, 40, 16, 5, 72, 20, 289, 45, 545, 68, 6467, 678, 2])

"a<s> casa ' quein no encerrações sobre paras como que Hillary temceri Figueirense Marta</s>"

In [15]:
sp.id_to_piece(1)

'<s>'

In [16]:
sp.encode_as_ids("<s> Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral</s>")

[3,
 1,
 160,
 22,
 20,
 158,
 23,
 733,
 63,
 30,
 13,
 6,
 78,
 20,
 319,
 45,
 624,
 72,
 3,
 63,
 200,
 752,
 2]

In [22]:
titles_dataset = tf.data.Dataset.from_tensor_slices(titles_as_id)

In [23]:
seq_len = 30
sequences = titles_dataset.batch(seq_len, drop_remainder=True)

In [24]:
def split_input_target(sentence):
    input_text = sentence[:-1]
    target_text = sentence[1:]
    return input_text, target_text

In [25]:
data_input_target = sequences.map(split_input_target)

In [26]:
for input_text, target_text in data_input_target.take(2):
    print(sp.decode_ids(input_text.numpy().tolist()))
    print(sp.decode_ids(target_text.numpy().tolist()))

<s> Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral</s> <s> 'Decidi ser escrava das mulheres que sofrem', diz cantora Oumou Sangaré</s> <s> Três reportagens da Folha ganham
<s> Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral</s> <s> 'Decidi ser escrava das mulheres que sofrem', diz cantora Oumou Sangaré</s> <s> Três reportagens da Folha ganham P
rêmio Petrobras de Jornalismo</s> <s> Filme 'Star Wars: Os  ⁇ ltimos Jedi' ganha trailer definitivo; assista</s> <s> CBSS inicia acordos com fintechs e quer 30% do crédito digital em 2018</s> <s> Em encontro, Bono pergunta a
Petrobras de Jornalismo</s> <s> Filme 'Star Wars: Os  ⁇ ltimos Jedi' ganha trailer definitivo; assista</s> <s> CBSS inicia acordos com fintechs e quer 30% do crédito digital em 2018</s> <s> Em encontro, Bono pergunta a Macri


In [27]:
BATCH_SIZE = 128

BUFFER_SIZE = 1000

batches = data_input_target.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

batches

<BatchDataset shapes: ((128, 59), (128, 59)), types: (tf.int32, tf.int32)>

# Build Model

In [28]:
vocab_size = sp.get_piece_size()

embedding_dim = 512

rnn_units = 1024

In [29]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 512)          4096000   
_________________________________________________________________
gru (GRU)                    (128, None, 1024)         4724736   
_________________________________________________________________
dense (Dense)                (128, None, 8000)         8200000   
Total params: 17,020,736
Trainable params: 17,020,736
Non-trainable params: 0
_________________________________________________________________


In [32]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [33]:
model.compile(optimizer='adam', loss=loss)

In [35]:
import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_latest")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [39]:
EPOCHS = 1
history = model.fit(batches, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 400 steps
400/400 [==============================] - 661s 2s/step - loss: 2.8326


# Restore model

In [40]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 512)            4096000   
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           4724736   
_________________________________________________________________
dense_1 (Dense)              (1, None, 8000)           8200000   
Total params: 17,020,736
Trainable params: 17,020,736
Non-trainable params: 0
_________________________________________________________________


# Run Model

In [41]:
def generate_text(model, start_string):
    start_str_ids = sp.encode_as_ids(start_string)
    input_eval = start_str_ids
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated_id = []
    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    temperature = 0.4
    model.reset_states()
    while len(text_generated_id)==0 or (text_generated_id[-1] != 2 and len(text_generated_id)<40):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated_id.append(int(predicted_id))

    #return start_string + ' ' +(sp.decode_ids(text_generated_id))
    return sp.decode_ids(start_str_ids+text_generated_id)

In [47]:
for i in range(0, 10):
    print(generate_text(model, "Brasil"))

Brasil exigirá logística global sobre grãos das finais</s>
Brasil março recibirou R$ 117,5 bilhões</s>
Brasil está derrotado novamente após chuva e até 12 de dezembro</s>
Brasil exigirá acordos globais</s>
Brasil goleia assim como membro do COI se afastar da Copa</s>
Brasil patina novamente</s>
Brasil convence Indonésia Federal Federal</s>
Brasil ultrapassa mínima mínima sobre juros nos EUA</s>
Brasil garantirá terceirização pública</s>
Brasil avança sociais, mas país tem altas</s>


In [46]:
for i in range(0, 10):
    print(generate_text(model, "<s>"))

<s>Jorna desabafa adquire julgamento sobre mensalão tucano</s>
<s> Fabricanterrada atinge maior alta mensal em sete anos</s>
<s>MAMOSPSPSP MandamentosSP Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos Mandamentos
<s>CanalLBA Preto Preto Preto Preto Preto Preto Peixoto Mandamentoserbergerberg Mandamentoserbergerbergerbergerbergerberg Mandamentoserberg Mandamentos Mandamentoserbergerberg Mandamentos Mandamentoserbergerberg Mandamentoserberg Mandamentoserberg Mandamentos Mandamentos Mandamentos Mandamentoserbergerbergerberg
<s>piloto automático disparou por gás <s> Conflito no Oriente Médio cai para 44o e deixam 46 feridos</s>
<s>Vocêdice investirá iPhone começa a cair d

In [21]:
sp.id_to_piece(5278)

'▁descumpr'

In [11]:
sp.encode_as_ids("")

[]